In [10]:
from netCDF4 import Dataset
import numpy as np
from tqdm.autonotebook import tqdm
import xarray

In [4]:
def AreaCalc(sizeLat,sizeLon): #Calculate surface area of grid cells
    deg2rd = np.pi/180.
    r=6378.1
    lon_bins = np.linspace(0,360., sizeLon+1)
    lat_bins = np.linspace(-80, 80, sizeLat+1) 
    Area=np.array([[deg2rd*(lon_bins[i+1]-lon_bins[i])*(np.sin(deg2rd*lat_bins[j+1])
                            - np.sin(deg2rd*lat_bins[j])) for i in range(len(lon_bins)-1)] 
                            for j in range(len(lat_bins)-1)])
    Area=r*r*Area*1000*1000 #convert it to m^2 instead of km^2
    return Area

In [5]:
def HistogramFunction(londata,latdata):
    londata,latdata=londata.reshape(np.size(londata)),latdata.reshape(np.size(latdata))
    binsLon=np.arange(-180,180)
    binsLat=np.arange(-80,80)
    density=np.zeros((len(binsLon),len(binsLat)))
    for i in range(np.array(londata).shape[0]):
        density[np.argmin(np.abs(londata[i]-binsLon)),np.argmin(np.abs(latdata[i]-binsLat))]+=1
    #Now, normalize it by area
    area=AreaCalc(len(binsLat),len(binsLon)).T
    density/=area
    density[density==0]=np.nan
    return density

In [12]:
dataset=Dataset('/Volumes/4YP/Data/Trajectories/Global_Total.nc')
lat=dataset.variables['lat'][:]
lon=dataset.variables['lon'][:]
time=dataset.variables['time'][:]
lon[lon>180]-=360
#Now, we want the last 5 years of particle positions, since in this time
#the garbage patch has largely been formed
if lon.shape[1]==4748:
    lonLast=lon[:,-365*5:]
    latLast=lat[:,-365*5:]
    Time=time[:,-365*5:]
else:
    lonLast=lon[:,-183*5:]
    latLast=lat[:,-183*5:]
    Time=time[:,-183*5:]
density=np.zeros((lonLast.shape[1],360,160))
for j in tqdm(range(lonLast.shape[1])):
    density[j,:,:]=HistogramFunction(lonLast[:,j],latLast[:,j])
density.dump('/Volumes/4YP/Data/Densities/Density_5Years_Global_Total')

/Users/jakecunningham/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in greater
  """


In [17]:
print(np.load('/Volumes/4YP/Data/Densities/Density_5Years_Global_Total', allow_pickle=True).shape)

(915, 360, 160)


In [18]:
print(np.load('/Volumes/4YP/Data/Densities/Density_Global_Total.npy', allow_pickle=True).shape)

(2374, 361, 161)


In [ ]:
file